# 描述符


## 1. how to
## [https://docs.python.org/zh-cn/3.9/howto/descriptor.html](https://docs.python.org/zh-cn/3.9/howto/descriptor.html)

### 1.1 入门

In [2]:
# The Ten class is a descriptor that always returns the constant 10 from its __get__() method:
class Ten:
    def __get__(self, obj, objtype=None):
        return 10

In [3]:
# To use the descriptor, it must be stored as a class variable in another class:
class A:
    x = 5
    y = Ten()      # Descriptor instance

In [4]:
a = A()
a.x    # Normal attribute lookup

5

In [5]:
a.y    # Descriptor lookup

10

In [6]:
# Dynamic lookups  动态查找
# 有趣的描述符通常运行计算而不是返回常量
import os

class DirectorySize:
    
    def __get__(self, obj, objtype=None):
        return len(os.listdir(obj.dirname))
    
class Directory:
    
    size = DirectorySize()    # Descriptor instance
    
    def __init__(self, dirname):
        self.dirname = dirname

In [7]:
s = Directory('songs')
g = Directory('games')

In [ ]:
s.size, g.size

In [9]:
# Managed attributes  托管/管理属性
import logging

logging.basicConfig(level=logging.INFO)

class LoggedAgeAccess:
    
    def __get__(self, obj, objtype=None):
        value = obj._age
        logging.info('Accessing %r giving %r', 'age', value)
        return value
    
    def __set__(self, obj, value):
        logging.info('Updating %r tp %r', 'age', value)
        obj._age = value
        
    
class Person:
    
    age = LoggedAgeAccess()    # Descriptor instance
    
    def __init__(self, name, age):
        self.name = name       # Ragular instance attribute
        self.age = age         # Calls __set__()
    
    def birthday(self):
        self.age += 1           # Calls both __get__() and __set__()


In [10]:
mary = Person('Mary M', 30)

INFO:root:Updating 'age' tp 30


In [11]:
dave = Person('David D', 40)

INFO:root:Updating 'age' tp 40


In [12]:
vars(mary)    # The actual data is in a private attribte

{'name': 'Mary M', '_age': 30}

In [13]:
mary.age

INFO:root:Accessing 'age' giving 30


30

In [14]:
mary.birthday()

INFO:root:Accessing 'age' giving 30
INFO:root:Updating 'age' tp 31


In [15]:
mary.age

INFO:root:Accessing 'age' giving 31


31

In [16]:
dave.name

'David D'

In [17]:
dave.age

INFO:root:Accessing 'age' giving 40


40

In [20]:
# Customized names    定制名称
import logging

logging.basicConfig(level=logging.INFO)

class LoggedAccess:
    
    def __set_name__(self, owner, name):
        self.public_name = name
        self.private_name = '_' + name
        
    def __get__(self, obj, objtype=None):
        value = getattr(obj, self.private_name)
        logging.info('Accessing %r giving %r', self.public_name, value)
        return value
    
    def __set__(self, obj, value):
        logging.info('Updating %r to %r', self.public_name, value)
        setattr(obj, self.private_name, value)
        
class Person:
    
    name = LoggedAccess()
    age = LoggedAccess()
    
    def __init__(self, name, age):
        self.name = name
        self.age = age
        
    def birthday(self):
        self.age += 1
        

In [22]:
vars(Person)

mappingproxy({'__module__': '__main__',
              'name': <__main__.LoggedAccess at 0x7fad7bc5cdc0>,
              'age': <__main__.LoggedAccess at 0x7fad7bc5cfa0>,
              '__init__': <function __main__.Person.__init__(self, name, age)>,
              'birthday': <function __main__.Person.birthday(self)>,
              '__dict__': <attribute '__dict__' of 'Person' objects>,
              '__weakref__': <attribute '__weakref__' of 'Person' objects>,
              '__doc__': None})

In [24]:
vars(Person)['name']

In [25]:
vars(vars(Person)['name'])

{'public_name': 'name', 'private_name': '_name'}

In [26]:
pete = Person('Perter P', 10)

INFO:root:Updating 'name' to 'Perter P'
INFO:root:Updating 'age' to 10


In [27]:
pete

In [28]:
vars(pete)

{'_name': 'Perter P', '_age': 10}

### 1.2 完整的实际例子

In [44]:
# 验证器类
from abc import ABC, abstractmethod

class Validator(ABC):
    
    def __set_name__(self, owner, name):
        self.private_name = '_' + name
        
    def __get__(self, obj, objtype=None):
        return getter(obj, self.private_name)
    
    def __set__(self, obj, value):
        self.validate(value)
        setattr(obj, self.private_name, value)
        
    @abstractmethod
    def validate(self, value):
        pass

# 自定义验证器
class OneOf(Validator):
    
    def __init__(self, *options):
        self.options = set(options)
        
    def validate(self, value):
        if value not in self.options:
            raise ValueError(f'Expected {value!r} to be one of {self.options!r}')
            
class Number(Validator):
    
    def __init__(self, minvalue=None, maxvalue=None):
        self.minvalue = minvalue
        self.maxvalue = maxvalue
        
    def validate(self, value):
        if not isinstance(value, (int, float)):
            raise TypeError(f'Expected {value!r} to be an int or float')
        if self.minvalue is not None and value < self.minvalue:
            raise ValueError(f'Expected {value!r} to be at least {self.minvalue!r}')
        if self.maxvalue is not None and value > self.maxvalue:
            raise ValueError(f'Expected {value!r} to be no more than {self.maxvalue!r}')
            
    
class String(Validator):
    
    def __init__(self, minsize=None, maxsize=None, predicate=None):
        self.minsize = minsize
        self.maxsize = maxsize
        self.predicate = predicate
        
    def validate(self, value):
        if not isinstance(value, str):
            raise TypeError(f'Expected {value!r} to be an str')
        if self.minsize is not None and len(value) < self.minsize:
            raise ValueError(f'Expected {value!r} to be no smaller than {self.minsize!r}')
        if self.maxsize is not None and len(value) > self.maxsize:
            raise ValueError(f'Expected {value!r} to be no bigger than {self.maxsize!r}')
        if self.predicate is not None and not self.predicate(value):
            raise ValueError(f'Expected {self.predicate} to be true for {value!r}')

In [45]:
# 验证器应用
class Component:
    
    name = String(minsize=3, maxsize=10, predicate=str.isupper)
    kind = OneOf('wood', 'metal', 'plastic')
    quantity = Number(minvalue=0)
    
    def __init__(self, name, kind, quantity):
        self.name = name
        self.kind = kind
        self.quantity = quantity
        

In [46]:
# 阻止无效实例的创建
Component('Widget', 'metal', 5)

ValueError: Expected <method 'isupper' of 'str' objects> to be true for 'Widget'

In [47]:
Component('WIDGET', 'metel', 5)

ValueError: Expected 'metel' to be one of {'wood', 'metal', 'plastic'}

In [48]:
Component('WIDGET', 'metal', -5)

ValueError: Expected -5 to be at least 0

In [49]:
Component('WIDGET', 'metal', 'V')

TypeError: Expected 'V' to be an int or float

In [50]:
c = Component('WIDGET', 'metal', 5)

In [53]:
c, vars(c), c.__dict__

(<__main__.Component at 0x7fad7bc5c880>,
 {'_name': 'WIDGET', '_kind': 'metal', '_quantity': 5},
 {'_name': 'WIDGET', '_kind': 'metal', '_quantity': 5})

In [60]:
# ORM(对象关系映射)实例
class Field:
    
    def __set_name__(self, owner, name):
        self.fetch = f'SELECT {name} FROM {owner.table} WHERE {owner.key}=?;'
        self.store = f'UPDATE {owner.table} SET {name}=? WHERE {owner.key}=?;'
        print('self.fetch', self.fetch)
        print('self.store', self.store)
        
    def __get__(self, obj, objtype=None):
        return conn.execute(self.fetch, [obj.key]).fetchone()[0]
    
    def __set__(self, obj, value):
        conn.execute(self.store, [value, obj.key])
        conn.commit()
        
    
class Movie:
    table = 'Movies'    # Table name
    key = 'title'       # Primary key
    director = Field()
    year = Field()
    
    def __init__(self, key):
        self.key = key
        

class Song:
    table = 'Music'
    key = 'title'
    artist = Field()
    year = Field()
    genre = Field()
    
    def __init__(self, key):
        self.key = key

self.fetch SELECT director FROM Movies WHERE title=?;
self.store UPDATE Movies SET director=? WHERE title=?;
self.fetch SELECT year FROM Movies WHERE title=?;
self.store UPDATE Movies SET year=? WHERE title=?;
self.fetch SELECT artist FROM Music WHERE title=?;
self.store UPDATE Music SET artist=? WHERE title=?;
self.fetch SELECT year FROM Music WHERE title=?;
self.store UPDATE Music SET year=? WHERE title=?;
self.fetch SELECT genre FROM Music WHERE title=?;
self.store UPDATE Music SET genre=? WHERE title=?;


In [56]:
import sqlite3
conn = sqlite3.connect('entertainment.db')

In [59]:
Movie('Start Wars').director

OperationalError: no such table: Movies

### 1.4 纯Python等价实现

property  属性/特性
Calling property()是构造数据描述符的简洁方式，该数据描述符在访问属性时触发函数调用。它的签名是
> property(fget=None, fset=None, fdel=None, doc=None) -> property

In [61]:
# 定义一个托管属性x的典型用法
class C:
    def getx(self): return self.__x
    def setx(self, value): self.__x = value
    def delx(self): del self.__x
    x = property(getx, setx, delx, "I'm the 'x' property.")

In [62]:
# <property>的python实现
class Property:
    "Emulate PyProperty_Type() in Objects/descrobject.c"
    
    def __init__(self, fget=None, fset=None, fdel=None, doc=None):
        self.fget = fget
        self.fset = fset
        self.fdel = fdel
        if doc is None and fget is not None:
            doc = fget.__doc__
        self.__doc__ = doc
        
    def __get__(self, obj, objtype=None):
        if obj is None:
            return self
        if self.fget is None:
            raise AttributeError("unreadable attribute")
        return self.fget(obj)
    
    def __set__(self, obj, value):
        if self.fset is None:
            raise AttributeError("can't set attribute")
        self.fset(obj, value)
    
    def __delete__(self, obj):
        if self.fdel is None:
            raise AttributeError("can't delete attribute")
        self.fdel(obj)
        
    def getter(self, fget):
        return type(self)(fget, self.fset, self.fdel, self.__doc__)
    
    def setter(self, fset):
        return type(self)(self.fget, fset, self.fdel, self.__doc__)
    
    def deleter(self, fdel):
        return type(self)(self.fget, self.fset, fdel, self.__doc__)
    
    

Python的面向对象功能是在基于函数的环境构建的，使用非数据描述符。

In [75]:
# 使用types.MethodType手动创建方法
class MethodType:
    "Emulate PyMethod_Type in Objects/classobject.c"
    
    def __init__(self, func, obj):
        self.__func__ = func
        self.__self__ = obj
        
    def __call__(self, *args, **kwargs):
        func = self.__func__
        obj = self.__self__
        return func(obj, *args, **kwargs)
    
class Function:
    
    def __get__(self, obj, objtype=None):
        "Simulate func_descr_get() in Objects/funcobject.c"
        print('__get__', ojb, objtype)
        if obj is None:
            return self
        return MethodType(self, obj)

In [76]:
class D:
    def f(self, x):
        return x


In [77]:
D.f, D.f.__qualname__    # 该函数具有qualified name(限定名称)属性以支持内省

(<function __main__.D.f(self, x)>, 'D.f')

In [78]:
vars(D)

mappingproxy({'__module__': '__main__',
              'f': <function __main__.D.f(self, x)>,
              '__dict__': <attribute '__dict__' of 'D' objects>,
              '__weakref__': <attribute '__weakref__' of 'D' objects>,
              '__doc__': None})

In [79]:
D.__dict__    # 通过类字典访问函数不会调用__get__(), 直接返回底层的函数

mappingproxy({'__module__': '__main__',
              'f': <function __main__.D.f(self, x)>,
              '__dict__': <attribute '__dict__' of 'D' objects>,
              '__weakref__': <attribute '__weakref__' of 'D' objects>,
              '__doc__': None})

In [80]:
d = D()
d.f    # 从实例通过.运算符访问，.运算符查找调用__get__(), 它返回一个绑定的方法对象

<bound method D.f of <__main__.D object at 0x7fad7bc8bd00>>

In [81]:
d.f.__func__

<function __main__.D.f(self, x)>

In [82]:
d.f.__self__

非数据描述符为"绑定函数到方法的通常模式"转变提供了一个简单的机制
Non-data descriptors provide a simple mechanism for variations on the usual patterns of binding functions into methods.

To recap, functions have a __get__() method so that they can be converted to a method when accessed as attributes. The non-data descriptor transforms an obj.f(*args) call into f(obj, *args). Calling cls.f(*args) becomes f(*args).

In [83]:
# Python版本的staticmethod()
class StaticMethod:
    "Emulate PyStaticMethod_Type() in Objects/funcobject.c"
    
    def __init__(self, f):
        self.f = f
        
    def __get__(self, obj, objtype=None):
        return self.f

In [84]:
# One use for class methods is to create alternate class constructors.
# dict.fromkeys() creates a new dictionary from a list of keys.
class Dict(dict):
    @classmethod
    def fromkeys(cls, iterable, value=None):
        "Emulate dict_fromkeys() in Objects/dictobject.c"
        d = cls()
        for key in iterable:
            d[key] = value
        return d

In [85]:
d = Dict.fromkeys('abracadabra')

In [86]:
type(d)

__main__.Dict

In [87]:
type(d) is Dict

True

In [88]:
d

{'a': None, 'b': None, 'r': None, 'c': None, 'd': None}

In [89]:
# Python版本的classmethod()
class ClassMethod:
    "Emulate PyClassMethod_Type() in Objects/funcobject.c"
    
    def __init__(self, f):
        self.f = f
        
    def __get__(self, obj, cls=None):
        if cls is None:
            cls = type(obj)
        # before Python 3.9
        def newfunc(*args):
            return self.f(cls, *args)
        return newfunc
        # Python 3.9, hasattr(obj, '__get__') was added, and make it possible for classmethod() to support chained decorators
        if hasattr(obj, '__get__'):
            return self.f.__get__(cls)
        return MethodType(self.f, cls)

In [90]:
# functools.cached_property()要求一个实例__dict__

from functools import cached_property

class CP:
    __slots__ = ()    # Eliminates the instance dict
    
    @cached_property  # Requires an instance dict
    def pi(self):
        return 4 * sum((-1.0)**n / (2.0*n + 1.0) for n in reversed(range(100_000)))

In [91]:
CP.pi

In [92]:
CP().pi

TypeError: No '__dict__' attribute on 'CP' instance to cache 'pi' property.

In [111]:
# Python版本的slot实现
# 用一个私有_slotvalues来模拟C结构体实现, 对该结构对读写通过“member descriptors”来管理
null = object()

class Member:
    
    def __init__(self, name, clsname, offset):
        "Emulate PyMemberDef in Include/structmember.h"
        # also see descr_new() in Objects/descrobject.c
        self.name = name
        self.clsname = clsname
        self.offset = offset
        
    def __get__(self, obj, objtype=None):
        "Emulate member_get() in Objects/descrobject.c"
        # also see PyMember_GetOne() in Python/structmember.c
        value = obj._slotvalues[self.offset]
        if value is null:
            raise AttributeError(self.name)
        return value
    
    def __set__(self, obj, value):
        "Emulate member_set() in Objects/descrobject.c"
        obj._slotvalues[self.offset] = value
        
    def __delete__(self, obj):
        "Emulate member_delete() in Objects/descrobject.c"
        value = obj._slotvalues[self.offset]
        if value is null:
            raise AttributeError(self.name)
        obj.__slotvalues[self.offset] = null
        
    def __repr__(self):
        "Emulate member_repr() in Objects/descrobject.c"
        return f'<Member {self.name!r} or {self.clsname!r}>'

In [112]:
# The type.__new__()方法来管理添加member对象到类变量
class Type(type):
    "Simulate how the type metaclass adds member objects for slots"
    
    def __new__(mcls, clsname, bases, mapping):
        "Emuluate type_new() in Objects/typeobject.c"
        # type_new() calls PyTypeReady() which calls add_methods()
        slot_names = mapping.get('slot_names', [])
        for offset, name in enumerate(slot_names):
            mapping[name] = Member(name, clsname, offset)
        return type.__new__(mcls, clsname, bases, mapping)

In [113]:
# The object.__new__() 方法来管理创建有slots而不是dict的实例
class Object:
    "Simulate how object.__new__() allocates memory for __slots__"
    
    def __new__(cls, *args):
        "Emulate object_new() in Objects/typeobject.c"
        inst = super().__new__(cls)
        if hasattr(cls, 'slot_names'):
            empty_slots = [null] * len(cls.slot_names)
            object.__setattr__(inst, '_slotvalues', empty_slots)
        return inst
    
    def __setattr__(self, name, value):
        "Emulate _PyObject_GenericSetAttrWithDict() Object/objects.c"
        cls = type(self)
        if hasattr(cls, 'slot_names') and name not in cls.slot_names:
            raise AttributeError(f'{type(self).__name__!r} object has no attribute {name!r}')
        super().__setattr__(name, value)
    
    def __delattr__(self, name):
        "Emulate _PyObject_GenericSetAttrWithDict() Objects/object.c"
        cls = type(self)
        if hasattr(cls, 'slot_names') and name not in cls.slot_names:
            raise AttributeError(f'{type(self).__name__!r} object has no attribute {name!r}')
        super().__delattr__(name)
            

In [114]:
# 要在真实的类中使用这个模拟版，只需从 Object 继承并将 metaclass 设为 Type：
class H(Object, metaclass=Type):
    "Instance variables stored in slots"
    
    slot_names = ['x', 'y']
    
    def __init__(self, x, y):
        self.x = x
        self.y = y
        

In [115]:
H

__main__.H

In [116]:
vars(H)

mappingproxy({'__module__': '__main__',
              '__doc__': 'Instance variables stored in slots',
              'slot_names': ['x', 'y'],
              '__init__': <function __main__.H.__init__(self, x, y)>,
              'x': <Member 'x' or 'H'>,
              'y': <Member 'y' or 'H'>})

In [117]:
h = H(10, 20)

In [118]:
h, vars(h)

(<__main__.H at 0x7fad80c90b20>, {'_slotvalues': [10, 20]})

In [119]:
h.x = 55

In [120]:
h.__dict__

{'_slotvalues': [55, 20]}

In [121]:
vars(h)

{'_slotvalues': [55, 20]}

In [122]:
h.xz

AttributeError: 'H' object has no attribute 'xz'